# Check out how the channel might meander from the old radarlines to new

Interpolation of old and new radarline comes from 26_RADAR_interpolate_line2line_by_distance_downchan


In [15]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import sys
import time
import datetime as dt
import pandas as pd
import geopandas as gpd
import scipy as sp
from scipy import signal
from tqdm.notebook import tqdm
import rasterio as rio
from shapely.geometry import Point, LineString, Polygon, MultiPoint
import fiona
from shapely.ops import nearest_points

In [16]:
new_gpr = gpd.read_file('/Volumes/arc_04/FIELD_DATA/K8621920/RES/PROCESSED_LINES/picked_bed.shp')
old_gpr = gpd.read_file('/Volumes/arc_04/FIELD_DATA/K8621920/RES/PROCESSED_LINES/1516_gprlines.shp')
old_gpr['camp'] = 'KIS2' 
bed_gdf = pd.concat([new_gpr,old_gpr])
gdf = bed_gdf[bed_gdf.camp =='KIS2'].copy()
ngpr = new_gpr[new_gpr.camp =='KIS2'].copy()

In [17]:
x_old = np.load('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/channel_line2line_interpbydistold_points_x.npy')
y_old = np.load('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/channel_line2line_interpbydistold_points_y.npy')
it_old = np.load('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/channel_line2line_interpbydistold_ice_thickn.npy')

In [18]:
x_new = np.load('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/channel_line2line_interpbydist_points_x.npy')
y_new = np.load('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/channel_line2line_interpbydist_points_y.npy')
it_new = np.load('/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/channel_line2line_interpbydist_ice_thickn.npy')

# plot channel over REMA surface

In [19]:
#load REMA
nid1 = 159202
nid2 = 131226
df = gpd.read_file('/Users/home/whitefar/DATA/REMOTE_SENSING/REMA_2m_strips/REMA_Strip_Index_Rel1.shp')
path = f"/Volumes/arc_02/whitefar/DATA/REMOTE_SENSING/REMA_STRIPES/DIFFERENCES/REMA_{nid1}-{nid2}_diff.tif"
f = df.loc[nid1].acquisitio
t =df.loc[nid2].acquisitio
duration = (pd.to_datetime(t)-pd.to_datetime(f)).days/365.25
with rio.open(path) as image:
    im = image.read(1)
    im[im==-9999] = np.nan
    im = im / duration
with rio.open('/Volumes/arc_02/whitefar/DATA/REMOTE_SENSING/REMA_STRIPES/SETSM_WV01_20161109_1020010058134D00_10200100576C9100_seg1_2m_v1.0_dem.tif') as image:
    extent_REMA = [image.bounds[0],image.bounds[2],image.bounds[1],image.bounds[3]]
    im_REMA = image.read(1)/100

In [20]:
df.loc[nid1].fileurl

'http://data.pgc.umn.edu/elev/dem/setsm/REMA/geocell/v1.0/2m/s83w153/SETSM_WV02_20121224_103001001D86CA00_103001001D886000_seg1_2m_v1.0.tar.gz'

In [21]:
direc = '/Users/home/whitefar/DATA/FIELD_ANT_19/POST_FIELD/RADAR_MAP/interpolatedgrid_allchan'

xv = np.load(direc + 'x_grid' +'.npy')
yv = np.load(direc + 'y_grid' +'.npy')
thickness_interpolated = np.load(direc +'ice_thickness'+'.npy')
extent_l2l = np.load(direc +'extent.npy')
surf_out = np.load(direc +'surface.npy')

In [22]:
x = np.linspace(extent_REMA[0],extent_REMA[1],im_REMA.shape[1]).astype('int')
y = np.linspace(extent_REMA[3],extent_REMA[2],im_REMA.shape[0]).astype('int')

In [23]:
xmin, xmax, ymin, ymax = extent_l2l

In [24]:
x_i = np.argwhere((x>xmin) & (x< xmax)).flatten()[0]
x_j = np.argwhere((x>xmin) & (x< xmax)).flatten()[-1]
y_i = np.argwhere((y>ymin) & (y< ymax)).flatten()[0]
y_j = np.argwhere((y>ymin) & (y< ymax)).flatten()[-1]

In [25]:
x_c = np.linspace(extent_REMA[0],extent_REMA[1],im_REMA.shape[1]).astype('int')[x_i:x_j]
y_c = np.linspace(extent_REMA[3],extent_REMA[2],im_REMA.shape[0]).astype('int')[y_i:y_j]

In [26]:
xv_REMA, yv_REMA = np.meshgrid(x_c, y_c)

In [27]:
im_REMA_d = im_REMA[y_i:y_j,x_i:x_j]
im_REMA_d[im_REMA_d==-99.99] = np.nan
im_REMA_c = im_REMA_d*100
del im_REMA_d

## 1. REMA bullseye with contours of channel

In [28]:

%matplotlib qt
#rema
v=[0,1]    

extent_REMA1 = [image.bounds[0],image.bounds[2],image.bounds[1],image.bounds[3]]
plt.figure(figsize=(15,15))
GyRd = plt.cm.get_cmap('RdGy').reversed()
plt.imshow(im,extent=extent_REMA1, vmin=v[0],vmax=v[1],cmap=GyRd)
cb = plt.colorbar()
cb.set_label('rate of elevation change, m/a')

# manual_locations = [(-380000, -727585),(-378000, -731200), (-380250,727650), (-377116, -729519), (-377640, -731742),  (-377470,-728870), (2.4, 1.7)]
manual_locations = [(-380000, -727585),(-378000, -731200),(-377640, -731742)]

CS = plt.contour(xv,yv,surf_out-thickness_interpolated)
plt.clabel(CS, inline=1, fontsize=8, fmt='%i', manual=manual_locations)


# cb2 = plt.colorbar()
# cb2.set_label('Elevation of ice underside, m')
plt.xlim([-382000, -374800])
plt.ylim([-733300,-724000])
plt.grid()
plt.show()

## 2. colors of chan with contours of surf

In [22]:
jj = 40

plt.figure(figsize=(10,15))

CS =  plt.contour(xv_REMA[::jj],yv_REMA[::jj],im_REMA_c[::jj],30)
plt.clabel(CS, inline=1, fontsize=8, fmt='%i')

v = [-750,-350]
plt.imshow(surf_out-thickness_interpolated,extent=extent_l2l,cmap='gist_rainbow')


cb = plt.colorbar()
cb.set_label('Elevation of ice base, m')
# plt.title('Colored base of ice with contours of surface topography')
plt.show()

## 3. thickness over length

In [15]:
it = it_new[10,:]
x = x_new[10,:]
y = y_new[10,:]
mid = pd.DataFrame({'x': x,'y':y,'ice_thickn':it})

points_mid = [Point(xy) for xy in zip(mid.x,mid.y)]
gdf_mid = gpd.GeoDataFrame(mid,geometry=points_mid,crs="EPSG:3031")

In [17]:
%matplotlib qt
#  fug, (a0, a1) = plt.subplots(2,1, gridspec_kw = {'height_ratios':[2, 3]},figsize=(6,6),dpi=180)#rema

fig, (ax1, ax2) = plt.subplots(1,2, sharey=True,gridspec_kw = {'width_ratios':[1, 4],'height_ratios':[1]},
                               figsize=(8,10),dpi=180)#rema ,constrained_layout=True
v=[0,1]    

kis2 = [-380737.537026,-724683.311299]

ax2.imshow(im_REMA,extent=extent_REMA,vmin=0.2,vmax=0.7, cmap='BrBG')
# cb = plt.colorbar()
# cb.set_label('surface elevation (?)')
# cb.set_label('rate of elevation change, m/a')
ax2.plot( gdf_mid.x,gdf_mid.y)
ax2.set_xlim([-382000, -374800])
ax2.set_ylim([-733300,-724000])
ax2.grid()
ax2.set_xlabel('GMT x, m')
# plt.contour(xv,yv,surf_out-thickness_interpolated,linewidths=0.75)


ax1.plot(gdf_mid.ice_thickn,gdf_mid.y)
# ax1.set_xlim([400, 480])
ax1.set_ylim([-733300,-724000])
ax1.set_ylabel('GMT y, m')
ax1.set_xlabel('ice thickness, m')

ax1.grid()
fig.show()

